# Yolo V4 Discovery
This notebook is to play with the sample code on Colab

Right now this:

    * generates the weights on the nhd-chicken google drive under My Drive/yolov4

## Warning

This notebook relies on weights that are stored on nhdchicken's google drive and therefore requires its credential to mount it.

In [20]:
%%bash
# This is some boiler plate code that clones the repository on Colab
if [  -d "/content/nhd-colab" ]
then
   echo "Environment already initialized"
elif [ -d "/content" ] && [ ! -d "/content/nhd-colab" ]
then
    cd /content || exit 1;
    echo "Installing https://github.com/nhdchicken/nhd-colab.git"
    git clone --recurse-submodules https://github.com/nhdchicken/nhd-colab.git || exit 1;
    cd nhd-colab || exit 1;
else
    echo "Not running in Colab - going to root of repos"
    cd `git rev-parse --show-toplevel` || exit 1;
fi
echo "Installing colab installer (which will run in the next cell)"
pip install utils/colab_install/ > /dev/null 2>&1 || exit 1;
echo "Checkin that Colab is available"
colab > /dev/null 2>&1 || exit 1;
echo "Great Success!"

Not running in Colab - going to root of repos
Installing colab installer (which will run in the next cell)
Checkin that Colab is available
Great Success!


In [22]:
# This line instal the yolov4-tf components 
! colab init yolov4-tf


running TensorFlow Version 2.1.0
going to repos root dir /Users/lpbrac/gitlab/pyops/nhd/nhd-colab
loading install config /Users/lpbrac/gitlab/pyops/nhd/nhd-colab/install.yml
installing components
yolov4-tf already installed - skipping


In [30]:
# This cell goes to the repository root of nhd-colab and sets the NHD_COLAB_REPOS_ROOT variable.
import os
import pathlib
import subprocess
if os.getcwd() == '/content':
  print("Running in Colab")
  NHD_COLAB_REPOS_ROOT = pathlib.Path(os.getcwd()) / 'nhd-colab'
else:
  print("Finding root of git repository")
  NHD_COLAB_REPOS_ROOT = pathlib.Path(subprocess.check_output('git rev-parse --show-toplevel', shell=True).decode('utf-8').strip())
os.chdir(NHD_COLAB_REPOS_ROOT)
assert os.getcwd() == str(NHD_COLAB_REPOS_ROOT.cwd())
assert os.getcwd().endswith('/nhd-colab')
print(f"NHD_COLAB_REPOS_ROOT={NHD_COLAB_REPOS_ROOT}\nOK!")

Finding root of git repository
NHD_COLAB_REPOS_ROOT=/Users/lpbrac/gitlab/pyops/nhd/nhd-colab
OK!


This cell:
    connects to the google drive
    converts weights to TF Lite with different quantization
    
    It set the following global variables:
    
    * YOLOV4_SUB_MOD_ROOT: The root of the YOLOV4 sub module
    * YOLOV4_WEIGHTS_DIR: Root directory where all weight are to be stored
    * YOLOV4_COCO_TFLITE_WEIGHTS: TF Lite weight
    * YOLOV4_COCO_TFLITE_FP16_WEIGHTS: 16 bits quantization
    * YOLOV4_COCO_TFLITE_UINT8_WEIGHTS: Unsigned 8 bit fixed point (UINT9)
    
Note that YOLOV4_COCO_TFLITE_UINT8_WEIGHTS fails in Tensor Flow right now
    
**ATTENTION**
Follow the instructions to connect the nhd-chicken google drive

In [51]:
""" When running on Colab, this cell attempt to mount the Google drive 
    to access the weights located at /content/drive/My Drive/yolov4

    If not in Colab, those weight are expected to be found under 
    nhd-colab/yolov4/tensorflow/data

    When not running in Colab, you can download the weight from 
    the Goodle Drive and store ir under the nhd-colab/yolov4/tensorflow/data
    directory or set the content in any directory and set the 
    YOLOV4_WEIGHTS_DIR environment variable to that location.
"""
import os
import warnings
import pathlib
import subprocess

YOLOV4_SUB_MOD_ROOT = NHD_COLAB_REPOS_ROOT / 'yolov4' / 'tensorflow'
YOLOV4_CONVERT_TFLITE_SCRIPT = 'convert_tflite.py '
YOLOV4_WEIGHTS_DIR = pathlib.Path("/content/drive/My Drive/yolov4")
COCO_DATA_SET_VAL = YOLOV4_SUB_MOD_ROOT / 'data' / 'dataset' / 'val2017.txt'
 
assert YOLOV4_SUB_MOD_ROOT.is_dir(), f"{YOLOV4_SUB_MOD_ROOT} not found"
try:
    from google.colab import drive
    drive.mount('/content/drive/')
except ModuleNotFoundError:
    print("Google drive not mounted since not running in Colab")
    YOLOV4_WEIGHTS_DIR = pathlib.Path(os.environ.get('YOLOV4_WEIGHTS_DIR', YOLOV4_SUB_MOD_ROOT / 'data'))

YOLOV4_COCO_TFLITE_WEIGHTS = YOLOV4_WEIGHTS_DIR / 'yolov4.tflite'
YOLOV4_COCO_TFLITE_FP16_WEIGHTS = YOLOV4_WEIGHTS_DIR / 'yolov4-fp16.tflite'
YOLOV4_COCO_TFLITE_UINT8_WEIGHTS = YOLOV4_WEIGHTS_DIR / 'yolov4-int8.tflite'

def convert(output, quant_mode=""):
    cmd = f"python3 {YOLOV4_CONVERT_TFLITE_SCRIPT} --weights '{YOLOV4_COCO_WEIGHTS}' --output '{output}' {quant_mode} --dataset {COCO_DATA_SET_VAL}"
    print(f"converting {YOLOV4_COCO_WEIGHTS} to  {output} ")
    print(f">> running command {cmd}")
    result = subprocess.run(cmd, stderr=subprocess.PIPE, shell=True)
    if result.returncode:
        raise OSError(f"command {cmd} failed\b {result.stderr.decode('utf-8')}")
    assert output.is_file(), f"{output} not found"
    print("OK!")
    
    
assert YOLOV4_WEIGHTS_DIR.is_dir(), f"YOLOV4_WEIGHTS_DIR={YOLOV4_WEIGHTS_DIR} not found!"
YOLOV4_COCO_WEIGHTS = YOLOV4_WEIGHTS_DIR / 'yolov4.weights'
assert YOLOV4_COCO_WEIGHTS.is_file(), f"YOLOV4_COCO_WEIGHTS={YOLOV4_COCO_WEIGHTS} not found!"
print(f"YOLOV4_COCO_WEIGHTS='{YOLOV4_COCO_WEIGHTS}")

# CONVERSION

CURR_DIR = os.getcwd()
os.chdir(YOLOV4_SUB_MOD_ROOT)
try:
    
    if not YOLOV4_COCO_TFLITE_WEIGHTS.is_file():
        convert(output=YOLOV4_COCO_TFLITE_WEIGHTS)
    assert YOLOV4_COCO_TFLITE_WEIGHTS.is_file(), f"{output} not found"
    
    if not YOLOV4_COCO_TFLITE_FP16_WEIGHTS.is_file():
        convert(output=YOLOV4_COCO_TFLITE_FP16_WEIGHTS, quant_mode='--quantize_mode float16')
    assert YOLOV4_COCO_TFLITE_FP16_WEIGHTS.is_file(), f"{output} not found"
        
    if not YOLOV4_COCO_TFLITE_UINT8_WEIGHTS.is_file():
        try:
          convert(output=YOLOV4_COCO_TFLITE_UINT8_WEIGHTS, quant_mode='--quantize_mode full_int8')
          assert YOLOV4_COCO_TFLITE_UINT8_WEIGHTS.is_file(), f"{output} not found"
        except:
          warnings.warn("conversion to UINT8 fails to be investigated")
    
except:
    raise
finally:
    os.chdir(CURR_DIR)

Google drive not mounted since not running in Colab
YOLOV4_COCO_WEIGHTS='/Users/lpbrac/gitlab/pyops/nhd/nhd-colab/yolov4/tensorflow/data/yolov4.weights
converting /Users/lpbrac/gitlab/pyops/nhd/nhd-colab/yolov4/tensorflow/data/yolov4.weights to  /Users/lpbrac/gitlab/pyops/nhd/nhd-colab/yolov4/tensorflow/data/yolov4-fp16.tflite 
>> running command python3 convert_tflite.py  --weights '/Users/lpbrac/gitlab/pyops/nhd/nhd-colab/yolov4/tensorflow/data/yolov4.weights' --output '/Users/lpbrac/gitlab/pyops/nhd/nhd-colab/yolov4/tensorflow/data/yolov4-fp16.tflite' --quantize_mode float16 --dataset /Users/lpbrac/gitlab/pyops/nhd/nhd-colab/yolov4/tensorflow/coco_dataset/coco/val207.txt
OK!
converting /Users/lpbrac/gitlab/pyops/nhd/nhd-colab/yolov4/tensorflow/data/yolov4.weights to  /Users/lpbrac/gitlab/pyops/nhd/nhd-colab/yolov4/tensorflow/data/yolov4-int8.tflite 
>> running command python3 convert_tflite.py  --weights '/Users/lpbrac/gitlab/pyops/nhd/nhd-colab/yolov4/tensorflow/data/yolov4.weights

OSError: command python3 convert_tflite.py  --weights '/Users/lpbrac/gitlab/pyops/nhd/nhd-colab/yolov4/tensorflow/data/yolov4.weights' --output '/Users/lpbrac/gitlab/pyops/nhd/nhd-colab/yolov4/tensorflow/data/yolov4-int8.tflite' --quantize_mode full_int8 --dataset /Users/lpbrac/gitlab/pyops/nhd/nhd-colab/yolov4/tensorflow/coco_dataset/coco/val207.txt failed 2020-06-17 19:22:26.752787: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-06-17 19:22:26.771281: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fd218084540 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-06-17 19:22:26.771294: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-06-17 19:22:33.818432: I tensorflow/core/grappler/devices.cc:60] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (